<a href="https://colab.research.google.com/github/toandaominh1997/Kaggle/blob/master/Review_Comment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import pandas
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
import pickle


def readTxt(filename):
  print(filename)
  lines = []
  with open(filename,encoding='utf-8', errors='ignore') as f:
    lines = f.readlines()
  
  return lines

neg = readTxt('neg_comment.txt')
pos = readTxt('pos_comment.txt')

y_neg = np.ones((len(neg), 1))
y_pos = np.zeros((len(pos), 1))
y_train = np.concatenate((y_neg, y_pos), axis=0)

neg.extend(pos)
all_text = neg
del neg
del pos
del y_neg
del y_pos
def Fit_Pre(all_text):
  word_vectorizer = TfidfVectorizer(
      sublinear_tf=True,
      strip_accents='unicode',
      encoding='utf-8',
      analyzer='word',
      token_pattern=r'\w{1,}',
      stop_words='english',
      ngram_range=(1, 1),
      norm='l2',
      min_df=0,
      smooth_idf=False,
      max_features=5000)
  word_vectorizer.fit(all_text)
  pickle.dump(word_vectorizer, open('word.sav', 'wb'))
  #train_word_features = word_vectorizer.transform(train)
  char_vectorizer = TfidfVectorizer(
      sublinear_tf=True,
      strip_accents='unicode',
      analyzer='char',
      stop_words='english',
      ngram_range=(2, 6),
      norm='l2',
      min_df=0,
      smooth_idf=False,
      max_features=1500)
  char_vectorizer.fit(all_text)
  pickle.dump(char_vectorizer, open('char.sav', 'wb'))
  #train_char_features = char_vectorizer.transform(train)
  #train_features = hstack([train_char_features, train_word_features])
  #return train_features
Fit_Pre(all_text)
def preprocessing(data):
  word = pickle.load(open('word.sav', 'rb'))
  char = pickle.load(open('char.sav', 'rb'))
  word_features = word.transform(data)
  char_features = char.transform(data)
  features = hstack([char_features, word_features])
  return features
train_features = preprocessing(all_text)

neg_comment.txt
pos_comment.txt


In [0]:
X = train_features

y = y_train
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


## Model Logistic Regression


In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import time

def LogReg(X_train, y_train):
  log = LogisticRegression(C=13.0)
  log.fit(X_train, y_train)
  return log

t = time.time()
print("Fitting...")
model_logreg = LogReg(X_train, y_train)
print("Done!")
print("Time to train with LogisticRegression: ", time.time()-t)


Fitting...


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Done!
Time to train with LogisticRegression:  0.6657376289367676


In [6]:
pred_train_logreg = model_logreg.predict(X_train)
print('Training accuracy is {}'.format(accuracy_score(y_train, pred_train_logreg)))

pred_test_logreg = model_logreg.predict(X_val)
print('Testing accuracy is {}'.format(accuracy_score(y_val, pred_test_logreg)))


Training accuracy is 0.9987109249113761
Testing accuracy is 0.9800193361263294


In [7]:
data = list()
data.append('I guess I should have realized a smaller screen wouldn’t fit our needs so it’s reslly not a fault of Amazon.')
data_test = preprocessing(data)

print(model_logreg.predict(data_test))

[1.]


## Model 2: Support Vector Machine


In [8]:
from sklearn.svm import SVC

svm = SVC(C=13.0, gamma='auto')
svm.fit(X_train, y_train)


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(C=13.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [9]:

print('Training accuracy is {}'.format(accuracy_score(y_train, svm.predict(X_train))))

print('Testing accuracy is {}'.format(accuracy_score(y_val, svm.predict(X_val))))

Training accuracy is 0.7978569126651628
Testing accuracy is 0.8001933612632935


## Model: RandomForestClassifier

In [28]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(max_depth=None, random_state=24)
rf.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=24, verbose=0, warm_start=False)

In [29]:

print('Training accuracy is {}'.format(accuracy_score(y_train, rf.predict(X_train))))

print('Testing accuracy is {}'.format(accuracy_score(y_val, rf.predict(X_val))))

Training accuracy is 0.9964550435062842
Testing accuracy is 0.9352239767966484


## Model 3: Lightgbm

In [20]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel

def LightGBM(X_train, y_train):
  model = LogisticRegression(solver='sag')
  sfm = SelectFromModel(model, threshold=0.2)

  _y_train = np.asarray(y_train)
  _y_train = _y_train.ravel()
  train_sparse_matrix = sfm.fit_transform(X_train, _y_train)
  train_sparse_matrix, valid_sparse_matrix, y_train_lgb, y_valid = train_test_split(train_sparse_matrix, _y_train, test_size=0.05, random_state=144)

  d_train = lgb.Dataset(train_sparse_matrix, label=y_train_lgb)
  d_valid = lgb.Dataset(valid_sparse_matrix, label=y_valid)
  watchlist = [d_train, d_valid]
 
  params = {'learning_rate': 0.2,
              'application': 'binary',
              'num_leaves': 31,
              'verbosity': -1,
              'metric': 'auc',
              'data_random_seed': 2,
              'bagging_fraction': 0.8,
              'feature_fraction': 0.6,
              'nthread': 4,
              'lambda_l1': 1,
              'lambda_l2': 1}

  model = lgb.train(params, train_set=d_train, valid_sets=watchlist, verbose_eval=10)
  return model

#model_lgb = LightGBM(X, y)
model_lgb = lgb.LGBMClassifier()
model_lgb.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.1, max_depth=-1,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
        random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [21]:
print('Training accuracy is {}'.format(accuracy_score(y_train, model_lgb.predict(X_train))))

print('Testing accuracy is {}'.format(accuracy_score(y_val, model_lgb.predict(X_val))))

Training accuracy is 1.0
Testing accuracy is 0.9800193361263294


## Model 4:XGboost

In [0]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier

x_train, X_valid, y_train_xgb, y_valid = train_test_split(
        X, y, test_size=0.2, random_state=23)
def XGBoost(X_train, X_valid, y_train_xgb, y_valid):
  xgb_params = {'eta': 0.3, 
              'max_depth': 5, 
              'subsample': 0.8, 
              'colsample_bytree': 0.8, 
              'objective': 'binary:logistic', 
              'eval_metric': 'auc', 
              'seed': 23
             }
  d_train = xgb.DMatrix(X_train, y_train_xgb)
  d_valid = xgb.DMatrix(X_valid, y_valid)
  watchlist = [(d_valid, 'valid')]
  model = xgb.train(xgb_params, d_train, 200, watchlist, verbose_eval=False, early_stopping_rounds=30)
  return model

model_xgb = XGBoost(x_train, X_valid, y_train_xgb, y_valid)

In [23]:
d_train = xgb.DMatrix(X_train)
pred_train_xgb = model_xgb.predict(d_train)
print('Training accuracy is {}'.format(accuracy_score(y_train, pred_train_xgb.round())))

d_test = xgb.DMatrix(X_val)
pred_test_xgb = model_xgb.predict(d_test)
print('Testing accuracy is {}'.format(accuracy_score(y_val, pred_test_xgb.round())))

Training accuracy is 0.9957299387689333
Testing accuracy is 0.9945214308733483


In [34]:
from sklearn.model_selection import StratifiedKFold
class StackModel():
  def __init__(self):
    self.model_lr = LogisticRegression(C=13.0)
    self.model_rf = RandomForestClassifier()
    self.model_xgb = xgb.XGBClassifier()
    self.model_lgb = lgb.LGBMClassifier()
    
  def fit(self, X_train, y_train):
    self.model_lr.fit(X_train, y_train)
    self.model_rf.fit(X_train, y_train)
    self.model_xgb.fit(X_train, y_train)
    self.model_lgb.fit(X_train, y_train)
      
  def predict(self, X):
    y_pred = 0.2*self.model_lr.predict(X) + 0.2*self.model_rf.predict(X) + 0.3*self.model_xgb.predict(X) + 0.3*self.model_lgb.predict(X)
    return np.round(y_pred)

model = StackModel()
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=24)
model.fit(X_train, y_train)

  

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-pa

In [35]:
print('Training accuracy is {}'.format(accuracy_score(y_train, model.predict(X_train))))
print('Testing accuracy is {}'.format(accuracy_score(y_val, model.predict(X_val))))

Training accuracy is 0.9986303577183371
Testing accuracy is 0.97035127296165
